<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f11f0e2f835555fe07a82d4a0f0975581855b8130f7badb2879fab5c781af0b4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-04 14:15:55
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: 21.55 K (0.15%)
Current PnL: -24.31 L (-15.71%)
CY Booked + Current PnL: -10.02 L (-6.47%)
-------------------
Total profit:  1.27 L
Total loss:  -25.58 L
-------------------
Total Booked + Current PnL: 16.96 L (13.27%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.93%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 88.94 L (61.78%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.28,-7.50,8.13,0.02,13127.0,-13099.0,161460.0,146.09,69.0,M-SC,15.58,234.0,-1.00,1.16,40.44,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.11,-16.62,20.04,0.09,16840.0,-16747.0,84030.0,95.09,38.0,M-SC,2.76,253.0,-0.99,0.60,10.40,OX40N,NTT,DURABLES
43,ITC,452.00,0.32,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,43.0,X-LC,1.58,5.0,-0.15,1.41,3.63,X40,NTT,FMCG
35,ICICIGI,2252.93,0.59,5.85,13.55,20.19,25001.0,10192.0,184512.0,-16.96,48.0,X-MC,6.62,68.0,0.41,1.33,21.84,X40,ATH,INSURANCE
50,MASFIN,398.61,-3.56,-5.50,29.15,22.05,26991.0,-5385.0,92595.0,-18.70,48.0,H-SC,6.82,164.0,-0.20,0.67,34.59,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.99,-44.28,212.57,74.15,193766.0,-72451.0,91154.0,-264.07,43.0,H-SC,19.99,149.0,-0.37,0.66,16.84,XR,NTT,CHEMICALS
13,BSOFT,831.70,2.51,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,74.0,H-SC,5.56,151.0,-0.30,0.79,28.66,XR,ATH,IT
29,HAPPSTMNDS,1480.71,2.46,-36.41,190.68,84.85,155412.0,-46661.0,81504.0,-24.31,55.0,H-SC,14.02,147.0,-0.30,0.59,4.61,AR,ATH,IT
40,INDUSINDBK,1800.00,1.92,-36.27,108.53,32.89,53399.0,-28004.0,49202.0,-707.41,64.0,L-MC,6.21,259.0,-0.52,0.35,35.50,XR,NTT,BANKS
73,TCS,4389.97,1.49,-12.31,36.03,19.28,109303.0,-42590.0,303366.0,-23.62,69.0,X-LC,6.53,1.0,-0.39,2.18,12.15,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,-3.56,-5.50,29.15,22.05,26991.0,-5385.0,92595.0,-18.70,48.0,H-SC,6.82,164.0,-0.20,0.67,34.59,XR,ATH,FINANCE
44,JCHAC,2282.00,-3.26,-33.62,50.63,-0.01,42954.0,-42965.0,84840.0,17269.57,8.0,M-SC,0.97,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
69,SURYODAY,216.00,-2.31,-21.59,56.91,23.03,79909.0,-38658.0,140413.0,53.51,43.0,H-SC,10.04,167.0,-0.48,1.01,39.32,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-1.66,-63.44,173.68,0.07,87113.0,-87020.0,50157.0,1738.38,55.0,L-SC,1.73,268.0,-1.00,0.36,24.74,OX40N,NTT,JEWELLERY
22,DIXON,18931.72,-1.59,-11.41,37.24,21.58,46235.0,-15984.0,124155.0,-55.32,26.0,X-MC,5.73,56.0,-0.35,0.89,9.92,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,-0.58,0.46,47.94,48.63,96300.0,923.0,200877.0,-33.16,64.0,H-SC,14.73,136.0,0.01,1.45,12.07,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.07,-5.98,121.14,107.91,168154.0,-8834.0,138810.0,-23.39,33.0,M-SC,11.31,216.0,-0.05,1.00,0.07,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.28,-7.50,8.13,0.02,13127.0,-13099.0,161460.0,146.09,69.0,M-SC,15.58,234.0,-1.00,1.16,40.44,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.11,-16.62,20.04,0.09,16840.0,-16747.0,84030.0,95.09,38.0,M-SC,2.76,253.0,-0.99,0.60,10.40,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.88,-22.35,46.13,13.47,96598.0,-60264.0,209403.0,-68.12,36.0,H-SC,2.12,158.0,-0.62,1.51,6.48,XY24,NTT,PAINTS
66,SIS,528.00,0.15,-24.35,60.29,21.26,50642.0,-27035.0,83997.0,1981.35,47.0,H-SC,4.05,166.0,-0.53,0.60,13.49,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.75,5.25,63.67,72.27,113552.0,8896.0,178344.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.75,XR,NTT,IT
37,IEX,219.0,-0.58,0.46,47.94,48.63,96300.0,923.0,200877.0,-33.16,64.0,H-SC,14.73,136.0,0.01,1.45,12.07,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.75,5.25,63.67,72.27,113552.0,8896.0,178344.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.75,XR,NTT,IT
37,IEX,219.00,-0.58,0.46,47.94,48.63,96300.0,923.0,200877.0,-33.16,64.0,H-SC,14.73,136.0,0.01,1.45,12.07,XR,NTT,MISC
38,INDIAMART,4810.62,-1.17,-1.07,108.96,106.72,132950.0,-1319.0,122017.0,-51.01,36.0,H-SC,1.56,139.0,-0.01,0.88,22.06,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.21,-1.20,37.28,35.63,77190.0,-2519.0,207055.0,-14.97,43.0,H-MC,3.41,119.0,-0.03,1.49,15.27,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.07,-5.98,121.14,107.91,168154.0,-8834.0,138810.0,-23.39,33.0,M-SC,11.31,216.0,-0.05,1.00,0.07,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.53,-21.10,88.16,48.45,139464.0,-42312.0,158194.0,-26.75,12.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.99,-37.57,116.58,35.20,93641.0,-48347.0,80323.0,5.27,24.0,X-SC,14.40,92.0,-0.52,0.58,0.32,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.59,-11.41,37.24,21.58,46235.0,-15984.0,124155.0,-55.32,26.0,X-MC,5.73,56.0,-0.35,0.89,9.92,X40N,ATH,IT
8,AWL,485.00,0.20,-21.50,95.53,53.50,226299.0,-64863.0,236888.0,-61.97,28.0,X-MC,1.52,58.0,-0.29,1.70,5.22,XY24,NTT,FMCG
23,DMART,5391.45,0.19,-6.92,37.64,28.12,73718.0,-14557.0,195850.0,-19.69,30.0,X-LC,4.21,19.0,-0.20,1.41,15.07,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.10,-23.51,114.62,64.15,208608.0,-55951.0,182000.0,-55.44,42.0,X-SC,1.48,82.0,-0.27,1.31,2.09,XY24,BTT,CEMENT
8,AWL,485.0,0.20,-21.50,95.53,53.50,226299.0,-64863.0,236888.0,-61.97,28.0,X-MC,1.52,58.0,-0.29,1.70,5.22,XY24,NTT,FMCG
43,ITC,452.0,0.32,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,43.0,X-LC,1.58,5.0,-0.15,1.41,3.63,X40,NTT,FMCG
15,CAMS,4762.0,0.23,1.14,23.38,24.79,59557.0,2878.0,254734.0,-3.47,47.0,X-SC,1.91,86.0,0.05,1.83,24.55,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.43,1.77,28.69,30.97,51606.0,3135.0,179873.0,-22.18,50.0,X-MC,2.22,75.0,0.06,1.29,14.56,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.53,-21.10,88.16,48.45,139464.0,-42312.0,158194.0,-26.75,12.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-1.17,-30.94,106.83,42.84,47961.0,-20111.0,44895.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
11,BATAINDIA,2096.00,-0.99,-37.57,116.58,35.20,93641.0,-48347.0,80323.0,5.27,24.0,X-SC,14.40,92.0,-0.52,0.58,0.32,X40,NTT,FOOTWEAR
54,PGHH,17907.65,0.47,-5.15,41.02,33.76,78137.0,-10335.0,190485.0,-33.05,34.0,X-MC,4.65,57.0,-0.13,1.37,0.47,X40,ATH,FMCG
75,TMPV,600.00,-0.36,-15.75,68.75,42.18,160108.0,-43525.0,232885.0,-24.30,31.0,X-LC,3.56,3.0,-0.27,1.68,0.88,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.17,-30.94,106.83,42.84,47961.0,-20111.0,44895.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,0.57,-47.10,187.92,52.31,144302.0,-68371.0,76789.0,-43.66,38.0,X-SC,5.34,91.0,-0.47,0.55,2.79,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.99,-37.57,116.58,35.20,93641.0,-48347.0,80323.0,5.27,24.0,X-SC,14.40,92.0,-0.52,0.58,0.32,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.59,-11.41,37.24,21.58,46235.0,-15984.0,124155.0,-55.32,26.0,X-MC,5.73,56.0,-0.35,0.89,9.92,X40N,ATH,IT
51,MEDANTA,1486.00,-1.16,0.06,24.01,24.08,30498.0,70.0,127020.0,-8.06,38.0,X-SC,5.93,89.0,0.00,0.91,19.68,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.49,-12.31,36.03,19.28,109303.0,-42590.0,303366.0,-23.62,69.0,X-LC,6.53,1.0,-0.39,2.18,12.15,X40,ATH,IT
41,INFY,2275.00,1.18,8.74,42.42,54.87,144332.0,27358.0,340246.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.45,17.75,X40,BTT,IT
75,TMPV,600.00,-0.36,-15.75,68.75,42.18,160108.0,-43525.0,232885.0,-24.30,31.0,X-LC,3.56,3.0,-0.27,1.68,0.88,XY24,NTT,AUTO
81,VBL,671.64,0.61,-2.96,39.81,35.67,122016.0,-9347.0,306495.0,-14.52,60.0,X-LC,5.30,4.0,-0.08,2.20,10.49,X40N,ATH,FMCG
43,ITC,452.00,0.32,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,43.0,X-LC,1.58,5.0,-0.15,1.41,3.63,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.92,-36.27,108.53,32.89,53399.0,-28004.0,49202.0,-707.41,64.0,L-MC,6.21,259.0,-0.52,0.35,35.50,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.69,-15.66,115.41,81.67,91310.0,-14692.0,79118.0,7111.11,42.0,L-SC,16.37,271.0,-0.16,0.57,53.22,XR,NTT,CERAMICS
50,MASFIN,398.61,-3.56,-5.50,29.15,22.05,26991.0,-5385.0,92595.0,-18.70,48.0,H-SC,6.82,164.0,-0.20,0.67,34.59,XR,ATH,FINANCE
13,BSOFT,831.70,2.51,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,74.0,H-SC,5.56,151.0,-0.30,0.79,28.66,XR,ATH,IT
69,SURYODAY,216.00,-2.31,-21.59,56.91,23.03,79909.0,-38658.0,140413.0,53.51,43.0,H-SC,10.04,167.0,-0.48,1.01,39.32,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.28,-7.50,8.13,0.02,13127.0,-13099.0,161460.0,146.09,69.0,M-SC,15.58,234.0,-1.00,1.16,40.44,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.12,3.35,17.02,20.95,44287.0,8439.0,260207.0,2.32,79.0,X-LC,17.93,31.0,0.19,1.87,40.18,X40,ATH,PAINTS
13,BSOFT,831.70,2.51,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,74.0,H-SC,5.56,151.0,-0.30,0.79,28.66,XR,ATH,IT
40,INDUSINDBK,1800.00,1.92,-36.27,108.53,32.89,53399.0,-28004.0,49202.0,-707.41,64.0,L-MC,6.21,259.0,-0.52,0.35,35.50,XR,NTT,BANKS
41,INFY,2275.00,1.18,8.74,42.42,54.87,144332.0,27358.0,340246.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.45,17.75,X40,BTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.35
1,25,45.27
2,50,76.17


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.19
MC    29.49
LC    24.29
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.02
X40      21.88
X40N     12.10
XR        9.91
AR        9.27
XY25      9.24
OX40N     7.71
SR        1.04
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.38
X-MC    22.89
X-LC    20.80
M-SC    12.09
X-SC     8.32
H-MC     4.86
L-SC     1.40
M-MC     1.39
M-LC     1.28
H-LC     1.20
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.07,-6.00,40.29
IT,12.51,-15.79,78.38
FINANCE,9.45,-18.02,67.89
MISC,7.55,-23.60,74.24
PAINTS,6.17,-7.72,23.72
ELECTRICAL,6.16,-9.63,49.79
INSURANCE,4.46,-1.95,37.88
PHARMA,3.98,-0.92,33.22
AUTO,3.41,-21.43,73.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3138984.0,21
XR,1285061.0,13
AR,1283621.0,10
X40,995159.0,14
X40N,765514.0,9
OX40N,719433.0,10
XY25,357337.0,6
SR,276789.0,2
MH,72454.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3591451.0,25
M-SC,1375663.0,15
X-MC,1298607.0,16
X-LC,871270.0,11
X-SC,763121.0,8
H-MC,403403.0,3
L-SC,266740.0,3
M-LC,113552.0,1
H-LC,66173.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1254384.0      6
           AR         884165.0      5
M-SC       XY24       784995.0      6
H-SC       XR         770640.0      7
X-MC       X40        451421.0      7
           XY24       385436.0      3
X-LC       X40        365109.0      5
H-SC       OX40N      333019.0      4
M-SC       OX40N      299301.0      5
X-SC       X40N       297425.0      3
           XY24       287067.0      3
H-SC       SR         276789.0      2
X-MC       X40N       272355.0      4
X-LC       XY24       233458.0      2
H-MC       AR         209759.0      2
X-LC       X40N       195734.0      2
H-MC       XY24       193644.0      1
X-MC       XY25       189395.0      2
L-SC       XR         179627.0      2
X-SC       X40        178629.0      2
M-SC       XR         167843.0      2
           AR         123524.0      2
M-LC       XR         113552.0      1
L-SC       OX40N       87113.0      1
X-LC       XY25        76969.0      2
H-SC       MH          72454.0      1
H-LC       AR          66173.0      1
L-MC       XR          53399.0      1
M-MC       XY25        51403.0      1
L-LC       XY25        39570.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
